In [1]:
import folium
import pandas as pd
import base64
from io import BytesIO

## Mapa do DF

In [2]:
mapa_df = folium.Map(location=[-15.7836, -47.7750])

In [3]:
mapa_df

In [4]:
mapa_df.save('maps/mapa-df.html')

## Mapa do Plano piloto

In [5]:
mapa_plano = folium.Map(location=[-15.792613, -47.882785],tiles='Stamen Toner',zoom_start=12)

In [6]:
mapa_plano

In [7]:
mapa_plano.save('maps/mapa-plano.html')

## Adicionando circulos para cada unidade

In [8]:
mapa_hospitais = folium.Map(location=[-15.7836, -47.7750],tiles='Stamen Toner')

In [9]:
df = pd.read_csv('data/hospitais-ubses-brasilia.csv')
for index, row in df.iterrows():
    folium.CircleMarker([row['lat'],row['long']],
                    radius=row['qtd_profissionais']/400,
                    popup=row['nome'],
                    color='#3186cc',
                    fill_color='#FFFFFF',
                    opacity=1,
                   ).add_to(mapa_hospitais)

In [10]:
mapa_hospitais

## Adicionando ícones

In [11]:
mapa_hospitais = folium.Map(location=[-15.7836, -47.7750])

### encodando imagem pois o ícone do folium só aceita URL

In [12]:
encoded = base64.b64encode(open('assets/red-hospital-256.png', 'rb').read())
decoded = base64.b64decode(encoded)
icon_url = BytesIO(decoded)

In [13]:
df = pd.read_csv('data/hospitais-ubses-brasilia.csv')
for index, row in df.iterrows():
    size = row['qtd_profissionais']/100
    icon_url = BytesIO(decoded)
    custom_pop = '<b>' + row['nome'] + '</b><br>' + 'Total de profissionais: '\
    + str(row['qtd_profissionais']) + '<br>' + 'Atendimento: ' +row['atendimento']\
    + '<br>' + 'Telefone: ' + row['telefone']
    marker = folium.Marker([row['lat'],row['long']],
                  popup=custom_pop,
                  icon=folium.CustomIcon(icon_url,icon_size=(size,size)),
                 ).add_to(mapa_hospitais)

In [14]:
mapa_hospitais

In [15]:
mapa_hospitais.save('maps/mapa-hospitais.html')

### testando ícones nativos do folium (awesome fonts)

In [16]:
mapa_teste = folium.Map(location=[-15.7836, -47.7750])

In [17]:
df = pd.read_csv('data/hospitais-ubses-brasilia.csv')
for index, row in df.iterrows():
    custom_pop = '<b>' + row['nome'] + '</b><br>' + 'Total de profissionais: '\
    + str(row['qtd_profissionais']) + '<br>' + 'Atendimento: ' +row['atendimento']\
    + '<br>' + 'Telefone: ' + row['telefone']
    marker = folium.Marker([row['lat'],row['long']],
                  popup=custom_pop,
                  icon=folium.Icon(color='red',icon='heart'),
                 ).add_to(mapa_teste)
#mapa_teste.add_child(folium.LatLngPopup())
mapa_teste